In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

# === Load Patients ===
fa_patients_path = "/home/snu/Downloads/NIFD_Patients_FA_Histogram_Feature.npy"
Patients_FA_array = np.load(fa_patients_path, allow_pickle=True)

# === Load Controls ===
fa_controls_path = "/home/snu/Downloads/NIFD_Control_FA_Histogram_Feature.npy"
Controls_FA_array = np.load(fa_controls_path, allow_pickle=True)

print("Patients Shape:", Patients_FA_array.shape)
print("Controls Shape:", Controls_FA_array.shape)

# === Combine features and labels ===
X = np.vstack([Controls_FA_array, Patients_FA_array])
y = np.hstack([
    np.zeros(Controls_FA_array.shape[0], dtype=np.int64),  # 0 = Control
    np.ones(Patients_FA_array.shape[0], dtype=np.int64)    # 1 = Patient
])

# Shuffle
np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]

Device: cuda
Patients Shape: (98, 180)
Controls Shape: (48, 180)


In [36]:
class ARMA_SemiSupervised(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device, activ="ELU"):
        super(ARMA_SemiSupervised, self).__init__()
        self.device = device
        self.conv1 = ARMAConv(input_dim, hidden_dim, num_stacks=1, num_layers=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.num_clusters = output_dim

        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu,
            "ELU": nnFn.elu
        }
        self.act = activations.get(activ, nnFn.elu)

    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x_in = x
      x = self.conv1(x, edge_index)
      x = self.bn1(x)

      if x_in.shape[1] == x.shape[1]:
          x = x + x_in  # residual (skip) connection is added

      x = self.act(x)
      x = self.dropout(x)
      logits = self.fc(x)
      return logits

    def cut_loss(self, A, S):
        S = nnFn.softmax(S, dim=1)
        A_pool = torch.matmul(torch.matmul(A, S).t(), S)
        num = torch.trace(A_pool)

        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, S).t(), S)
        den = torch.trace(D_pooled)
        mincut_loss = -(num / den)

        St_S = torch.matmul(S.t(), S)
        I_S = torch.eye(self.num_clusters, device=self.device)
        ortho_loss = torch.norm(St_S / torch.norm(St_S) - I_S / torch.norm(I_S))

        return mincut_loss + ortho_loss

In [38]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [40]:
num_nodes, num_feats = X.shape
print(f"Number of features: {num_feats}")

Number of features: 180


In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.5
feats_dim = num_feats
hidden_dim = 512
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 100
lambda_mod = 0.01 #0.01  # weight for modularity loss
# lambda_sup = 5

In [46]:
W = create_adj(X, alpha)
data = load_data(W, X).to(device)
A_tensor = torch.from_numpy(W).float().to(device)
print(data)

Data(x=[146, 180], edge_index=[2, 21256])


In [47]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

for fold, (train_val_idx, test_idx_global) in enumerate(sss.split(X, y), start=1):
    print(f"\n=== Fold {fold} ===")

    # Split into controls (label 0) and patients (label 1)
    controls_idx = np.where(y == 0)[0]
    patients_idx = np.where(y == 1)[0]

    sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=fold)
    controls_train_idx, _ = next(sss_class.split(X[controls_idx], y[controls_idx]))
    patients_train_idx, _ = next(sss_class.split(X[patients_idx], y[patients_idx]))

    controls_train = controls_idx[controls_train_idx]
    patients_train = patients_idx[patients_train_idx]
    train_idx_final = np.concatenate([controls_train, patients_train])
    np.random.shuffle(train_idx_final)

    print(f"Train Controls: {len(controls_train)}, Train Patients: {len(patients_train)}")

    train_idx_t = torch.from_numpy(train_idx_final).long().to(device)
    y_train_tensor = torch.from_numpy(y[train_idx_final]).long().to(device)

    # Initialize model and optimizer
    model = ARMA_SemiSupervised(feats_dim, hidden_dim, num_classes, device, "SELU").to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
        loss_unsup = model.cut_loss(A_tensor, logits)
        total_loss = loss_sup + lambda_mod * loss_unsup

        total_loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: "
                  f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                  f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

    y_test = y[test_idx_global]
    y_pred_test = preds[test_idx_global]
    y_prob_test = probs[test_idx_global]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")

# Summary across all folds
print("\n=== Average Results Across 20 Folds (Controls vs Patients) ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")



=== Fold 1 ===
Train Controls: 4, Train Patients: 9
Fold 1 Epoch 1: TotalLoss=0.912531 | Sup=0.914981 | Unsup=-0.245040 | TrainAcc=0.2308
Fold 1 Epoch 100: TotalLoss=0.020418 | Sup=0.023120 | Unsup=-0.270217 | TrainAcc=1.0000
Fold 1 Epoch 200: TotalLoss=0.005073 | Sup=0.007893 | Unsup=-0.281981 | TrainAcc=1.0000
Fold 1 Epoch 300: TotalLoss=0.001252 | Sup=0.004080 | Unsup=-0.282775 | TrainAcc=1.0000
Fold 1 Epoch 400: TotalLoss=-0.000719 | Sup=0.002262 | Unsup=-0.298111 | TrainAcc=1.0000
Fold 1 Epoch 500: TotalLoss=-0.001686 | Sup=0.001408 | Unsup=-0.309335 | TrainAcc=1.0000
Fold 1 Epoch 600: TotalLoss=-0.002711 | Sup=0.000613 | Unsup=-0.332408 | TrainAcc=1.0000
Fold 1 Epoch 700: TotalLoss=-0.003206 | Sup=0.000350 | Unsup=-0.355571 | TrainAcc=1.0000
Fold 1 Epoch 800: TotalLoss=-0.003750 | Sup=0.000183 | Unsup=-0.393329 | TrainAcc=1.0000
Fold 1 Epoch 900: TotalLoss=-0.003799 | Sup=0.000289 | Unsup=-0.408767 | TrainAcc=1.0000
Fold 1 Epoch 1000: TotalLoss=-0.004309 | Sup=0.000095 | Unsup=-

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

# ==========================================
# CONFIG
# ==========================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_dim = 512
num_epochs = 5000
lr = 1e-4
weight_decay = 1e-4
batch_print_freq = 500  # print every 500 epochs

# λ_mod list
lambda_mod_list = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]

# Directory for checkpoints
checkpoint_dir = "ARMA_Cut_NIFD"
os.makedirs(checkpoint_dir, exist_ok=True)

results_summary = []

# ==========================================
# Main loop for λ_mod sweep
# ==========================================
for lambda_mod in lambda_mod_list:
    print(f"\n==============================")
    print(f" Running with λ_mod = {lambda_mod}")
    print(f"==============================")

    accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []
    sss = StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=SEED)

    for fold, (train_val_idx, test_idx_global) in enumerate(sss.split(X, y), start=1):
        print(f"\n=== Fold {fold} ===")

        # --- Split Controls and Patients ---
        controls_idx = np.where(y == 0)[0]
        patients_idx = np.where(y == 1)[0]

        sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=fold)
        controls_train_idx, _ = next(sss_class.split(X[controls_idx], y[controls_idx]))
        patients_train_idx, _ = next(sss_class.split(X[patients_idx], y[patients_idx]))

        controls_train = controls_idx[controls_train_idx]
        patients_train = patients_idx[patients_train_idx]
        train_idx_final = np.concatenate([controls_train, patients_train])
        np.random.shuffle(train_idx_final)

        print(f"Train Controls: {len(controls_train)}, Train Patients: {len(patients_train)}")

        train_idx_t = torch.from_numpy(train_idx_final).long().to(device)
        y_train_tensor = torch.from_numpy(y[train_idx_final]).long().to(device)

        # --- Initialize model ---
        model = ARMA_SemiSupervised(feats_dim, hidden_dim, num_classes, device, "RELU").to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        ce_loss = nn.CrossEntropyLoss()

        best_loss = float('inf')
        best_model_state = None

        # --- Training loop ---
        for epoch in range(1, num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            logits = model(data)
            loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
            loss_unsup = model.cut_loss(A_tensor, logits)  # unsupervised regularization
            total_loss = loss_sup + lambda_mod * loss_unsup

            total_loss.backward()
            optimizer.step()

            # Track best model
            if total_loss.item() < best_loss:
                best_loss = total_loss.item()
                best_model_state = model.state_dict().copy()

            if epoch % batch_print_freq == 0 or epoch == 1:
                model.eval()
                with torch.no_grad():
                    preds_train = logits[train_idx_t].argmax(dim=1)
                    acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
                print(f"Fold {fold} Epoch {epoch}: "
                      f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                      f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

        # --- Save checkpoint ---
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_lambda_{lambda_mod}_fold_{fold}.pt")
        torch.save({
            'lambda_mod': lambda_mod,
            'fold': fold,
            'model_state_dict': best_model_state,
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_loss,
        }, checkpoint_path)
        print(f" Saved checkpoint: {checkpoint_path}")

        # --- Evaluation ---
        model.load_state_dict(best_model_state)
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out.argmax(dim=1).cpu().numpy()
            probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

        y_test = y[test_idx_global]
        y_pred_test = preds[test_idx_global]
        y_prob_test = probs[test_idx_global]

        acc = accuracy_score(y_test, y_pred_test)
        prec = precision_score(y_test, y_pred_test, zero_division=0)
        rec = recall_score(y_test, y_pred_test, zero_division=0)
        f1 = f1_score(y_test, y_pred_test, zero_division=0)
        auc = roc_auc_score(y_test, y_prob_test)
        ce = log_loss(y_test, y_prob_test)

        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
        aucs.append(auc)
        ce_losses.append(ce)

        print(f"Fold {fold} → "
              f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
              f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")

    # --- Average results per λ_mod ---
    mean_acc, std_acc = np.mean(accuracies), np.std(accuracies)
    mean_prec, std_prec = np.mean(precisions), np.std(precisions)
    mean_rec, std_rec = np.mean(recalls), np.std(recalls)
    mean_f1, std_f1 = np.mean(f1_scores), np.std(f1_scores)
    mean_auc, std_auc = np.mean(aucs), np.std(aucs)
    mean_ce, std_ce = np.mean(ce_losses), np.std(ce_losses)

    results_summary.append({
        "λ_mod": lambda_mod,
        "Accuracy": f"{mean_acc:.4f} ± {std_acc:.4f}",
        "Precision": f"{mean_prec:.4f} ± {std_prec:.4f}",
        "Recall": f"{mean_rec:.4f} ± {std_rec:.4f}",
        "F1": f"{mean_f1:.4f} ± {std_f1:.4f}",
        "AUC": f"{mean_auc:.4f} ± {std_auc:.4f}",
        "CE Loss": f"{mean_ce:.4f} ± {std_ce:.4f}",
    })

    print(f"\n=== λ_mod = {lambda_mod} → Average Results ===")
    print(f"Accuracy:  {mean_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision: {mean_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall:    {mean_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score:  {mean_f1:.4f} ± {std_f1:.4f}")
    print(f"AUC:       {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"CE Loss:   {mean_ce:.4f} ± {std_ce:.4f}")

# ==========================================
# Final summary table
# ==========================================
print("\n\n========== FINAL SUMMARY TABLE (Controls vs Patients) ==========")
results_df = pd.DataFrame(results_summary)
print(results_df.to_string(index=False))



 Running with λ_mod = 0.001

=== Fold 1 ===
Train Controls: 24, Train Patients: 49
Fold 1 Epoch 1: TotalLoss=0.861455 | Sup=0.861688 | Unsup=-0.232636 | TrainAcc=0.2877
Fold 1 Epoch 500: TotalLoss=0.006631 | Sup=0.006872 | Unsup=-0.240591 | TrainAcc=1.0000
Fold 1 Epoch 1000: TotalLoss=0.000850 | Sup=0.001110 | Unsup=-0.260094 | TrainAcc=1.0000
Fold 1 Epoch 1500: TotalLoss=-0.000022 | Sup=0.000330 | Unsup=-0.351255 | TrainAcc=1.0000
Fold 1 Epoch 2000: TotalLoss=-0.000305 | Sup=0.000143 | Unsup=-0.447872 | TrainAcc=1.0000
Fold 1 Epoch 2500: TotalLoss=-0.000358 | Sup=0.000112 | Unsup=-0.469697 | TrainAcc=1.0000
Fold 1 Epoch 3000: TotalLoss=-0.000417 | Sup=0.000064 | Unsup=-0.480532 | TrainAcc=1.0000
Fold 1 Epoch 3500: TotalLoss=-0.000436 | Sup=0.000045 | Unsup=-0.481451 | TrainAcc=1.0000
Fold 1 Epoch 4000: TotalLoss=-0.000410 | Sup=0.000073 | Unsup=-0.483228 | TrainAcc=1.0000
Fold 1 Epoch 4500: TotalLoss=-0.000419 | Sup=0.000063 | Unsup=-0.482046 | TrainAcc=1.0000
Fold 1 Epoch 5000: Tota

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

# ==========================================
# CONFIG
# ==========================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_dim = 512
num_epochs = 5000
lr = 1e-4
weight_decay = 1e-4
batch_print_freq = 500  # print every 500 epochs

# λ_mod list
lambda_mod_list = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]

# Directory for checkpoints
checkpoint_dir = "ARMA_Cut_NIFD_90_10"
os.makedirs(checkpoint_dir, exist_ok=True)

results_summary = []

# ==========================================
# Main loop for λ_mod sweep
# ==========================================
for lambda_mod in lambda_mod_list:
    print(f"\n==============================")
    print(f" Running with λ_mod = {lambda_mod}")
    print(f"==============================")

    accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []
    sss = StratifiedShuffleSplit(n_splits=20, test_size=0.1, random_state=SEED)

    for fold, (train_val_idx, test_idx_global) in enumerate(sss.split(X, y), start=1):
        print(f"\n=== Fold {fold} ===")

        # --- Split Controls and Patients ---
        controls_idx = np.where(y == 0)[0]
        patients_idx = np.where(y == 1)[0]

        sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.1, random_state=fold)
        controls_train_idx, _ = next(sss_class.split(X[controls_idx], y[controls_idx]))
        patients_train_idx, _ = next(sss_class.split(X[patients_idx], y[patients_idx]))

        controls_train = controls_idx[controls_train_idx]
        patients_train = patients_idx[patients_train_idx]
        train_idx_final = np.concatenate([controls_train, patients_train])
        np.random.shuffle(train_idx_final)

        print(f"Train Controls: {len(controls_train)}, Train Patients: {len(patients_train)}")

        train_idx_t = torch.from_numpy(train_idx_final).long().to(device)
        y_train_tensor = torch.from_numpy(y[train_idx_final]).long().to(device)

        # --- Initialize model ---
        model = ARMA_SemiSupervised(feats_dim, hidden_dim, num_classes, device, "RELU").to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        ce_loss = nn.CrossEntropyLoss()

        best_loss = float('inf')
        best_model_state = None

        # --- Training loop ---
        for epoch in range(1, num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            logits = model(data)
            loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
            loss_unsup = model.cut_loss(A_tensor, logits)  # unsupervised regularization
            total_loss = loss_sup + lambda_mod * loss_unsup

            total_loss.backward()
            optimizer.step()

            # Track best model
            if total_loss.item() < best_loss:
                best_loss = total_loss.item()
                best_model_state = model.state_dict().copy()

            if epoch % batch_print_freq == 0 or epoch == 1:
                model.eval()
                with torch.no_grad():
                    preds_train = logits[train_idx_t].argmax(dim=1)
                    acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
                print(f"Fold {fold} Epoch {epoch}: "
                      f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                      f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

        # --- Save checkpoint ---
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_lambda_{lambda_mod}_fold_{fold}.pt")
        torch.save({
            'lambda_mod': lambda_mod,
            'fold': fold,
            'model_state_dict': best_model_state,
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_loss,
        }, checkpoint_path)
        print(f" Saved checkpoint: {checkpoint_path}")

        # --- Evaluation ---
        model.load_state_dict(best_model_state)
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out.argmax(dim=1).cpu().numpy()
            probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

        y_test = y[test_idx_global]
        y_pred_test = preds[test_idx_global]
        y_prob_test = probs[test_idx_global]

        acc = accuracy_score(y_test, y_pred_test)
        prec = precision_score(y_test, y_pred_test, zero_division=0)
        rec = recall_score(y_test, y_pred_test, zero_division=0)
        f1 = f1_score(y_test, y_pred_test, zero_division=0)
        auc = roc_auc_score(y_test, y_prob_test)
        ce = log_loss(y_test, y_prob_test)

        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
        aucs.append(auc)
        ce_losses.append(ce)

        print(f"Fold {fold} → "
              f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
              f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")

    # --- Average results per λ_mod ---
    mean_acc, std_acc = np.mean(accuracies), np.std(accuracies)
    mean_prec, std_prec = np.mean(precisions), np.std(precisions)
    mean_rec, std_rec = np.mean(recalls), np.std(recalls)
    mean_f1, std_f1 = np.mean(f1_scores), np.std(f1_scores)
    mean_auc, std_auc = np.mean(aucs), np.std(aucs)
    mean_ce, std_ce = np.mean(ce_losses), np.std(ce_losses)

    results_summary.append({
        "λ_mod": lambda_mod,
        "Accuracy": f"{mean_acc:.4f} ± {std_acc:.4f}",
        "Precision": f"{mean_prec:.4f} ± {std_prec:.4f}",
        "Recall": f"{mean_rec:.4f} ± {std_rec:.4f}",
        "F1": f"{mean_f1:.4f} ± {std_f1:.4f}",
        "AUC": f"{mean_auc:.4f} ± {std_auc:.4f}",
        "CE Loss": f"{mean_ce:.4f} ± {std_ce:.4f}",
    })

    print(f"\n=== λ_mod = {lambda_mod} → Average Results ===")
    print(f"Accuracy:  {mean_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision: {mean_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall:    {mean_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score:  {mean_f1:.4f} ± {std_f1:.4f}")
    print(f"AUC:       {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"CE Loss:   {mean_ce:.4f} ± {std_ce:.4f}")

# ==========================================
# Final summary table
# ==========================================
print("\n\n========== FINAL SUMMARY TABLE (Controls vs Patients) ==========")
results_df = pd.DataFrame(results_summary)
print(results_df.to_string(index=False))
